# IMPORTING REQUIRED LIBRARIES:

BEFORE EXECUTING THE CODE, PLEASE CHECK WHETHER YOU HAVE THE FOLLOWING LIBRARIES:

1. functools - 
If not, use this command to install -  'pip3 install functools'
2. plotly.express - 
If not, follow the procesure given in the user manual.

In [1]:
import os
import pandas as pd
import numpy as np
from IPython.core.display import HTML
pd.set_option("display.max_columns",75)
import warnings
warnings.filterwarnings('ignore')
import pandas_profiling
from functools import reduce
import plotly.express as px
import operator
from collections import Counter 
from collections import OrderedDict

# READING AND OBTAINING DATE AND TIME FROM DATETIME STAMP:

- rest.csv file is loaded and stored inside data.
- Total Amount of the product sale is calculated by using Quantity * Product Price.
- Data is sorted according to the Order Numbers.
- Column 'Order Date' is converted to datetime and stored in Column 'date'.
- Column 'date' is parsed to obtain Date and Time seperately.
- For better analysis, we consider only data for a time span of 1 year i.e from 01 January 2017 to 01 January 2018.
- Some Outliers were removed so that we can obtain unbiased results.
- Column 'Order Date' is dropped.

In [2]:
%%time
data = pd.read_csv('D://Downloads//restaurant-1-orders.csv//rest.csv')
data['Amount'] = data['Quantity']* data['Product Price']
data = data.sort_values(by = 'Order Number',ascending=True)
data['date'] =  pd.to_datetime(data['Order Date'])
data.index = data['Order Date']
data['Time'],data['Date']= data['date'].apply(lambda x:x.time()), data['date'].apply(lambda x:x.date())
data = data.drop(['Order Date','date'],axis=1)
data['Date'] =  pd.to_datetime(data['Date'])
data = data[(data['Date']>'2017-01-01')& (data['Date']<'2018-01-01')]
data = data[(data['Order Number']!=6768) & (data['Order Number']!= 6769)]
data.reset_index(inplace=True)
data = data.drop('Order Date',axis=1)
data.head()

Wall time: 1.46 s


,Order Number,Item Name,Quantity,Product Price,Total products,Amount,Time,Date
0,4164,Keema Naan,1,2.95,4,2.95,18:01:00,2017-02-01
1,4164,Chicken Biryani,2,9.95,4,19.90,18:01:00,2017-02-01
2,4164,Madras,1,7.95,4,7.95,18:01:00,2017-02-01
3,4164,Pilau Rice,1,2.95,4,2.95,18:01:00,2017-02-01
4,4165,Saag Aloo,1,5.95,4,5.95,18:16:00,2017-02-01


# DATA PREPROCESSING AND MULTIPLE DATAFRAME CREATION:

- We have 177 distinct food items in the dataset.
- All the food items are categorised into sets such as mcveg(Main Course Veg),mcnonveg(Main Course Non-Veg),beverages and sidedish.
- We create dataframes with names 'orderdesc','ordertype','order_date','quantamt' which contain different sets of values in them.
- We then create two more major dataframes named 'complete_data' and 'complete_data1' which contain food items and quantity,amount and timezones respectively.


In [3]:
%%time
mcveg = ['Aloo Chaat','Aloo Gobi','Aloo Methi','Baingan Hari Mirch','Bhindi Bhajee','Bhuna',
 'Bombay Aloo','Brinjal Bhajee','Cauliflower Bhajee','Chana Masala','Curry',
 'Curry Sauce','Dall Samba','Dhansak','Dhansak Sauce','Madras','Madras Sauce','Mango Chutney','Masala Sauce','Methi',
 'Mint Sauce','Mixed Starter','Mixed Vegetable Curry','Mushroom','Mushroom Bhajee','Mushroom Rice',
 'Muttar Paneer','Onion Bhajee','Onion Chutney','Onion Naan','Onion Rice','Paneer Shaslick','Paneer Tikka Balti',
 'Paneer Tikka Karahi','Paneer Tikka Masala','Paneer Tikka Sizzler','Tarka Dall','Vegetable Balti','Vegetable Biryani',
 'Vegetable Karahi','Vegetable Mysore','Vegetable Rice','Vegetable Roll','Saag','Saag Aloo','Saag Bhajee','Saag Dall',
 'Saag Paneer',]
mcnonveg = ['Bengal Fish Biryani','Bengal Fish Karahi','Bengal Fry Fish','Bengal King Prawn','Butter Chicken','Chicken Achar',
 'Chicken Balti','Chicken Biryani','Chicken Chaat','Chicken Chilli Garlic','Chicken Hari Mirch','Chicken Haryali',
 'Chicken Karahi','Chicken Mysore','Chicken Pakora','Chicken Rezala','Chicken Roshni','Chicken Shashlick',
 'Chicken Shashlick Curry','Chicken Sylhet','Chicken Tikka','Chicken Tikka (Main)','Chicken Tikka Balti',
 'Chicken Tikka Biryani','Chicken Tikka Chilli Masala','Chicken Tikka Garlic','Chicken Tikka Jalfrezi',
 'Chicken Tikka Jeera','Chicken Tikka Karahi','Chicken Tikka Masala','Chicken Tikka Pasanda',
 'Chicken Tikka Sizzler','Dupiaza','Egg Paratha','Egg Rice','Hazary Lamb',
 'Hazary Lamb Chilli Garlic','King Prawn Balti','King Prawn Biryani','King Prawn Butterfly','King Prawn Hari Mirch',
 'King Prawn Karahi','King Prawn Puree','King Prawn Shaslick',
 'Korma','Korma Sauce','Lal Mirch Chicken','Lal Mirch Lamb','Lamb Achar','Lamb Balti','Lamb Biryani','Lamb Chilli Garlic',
 'Lamb Hari Mirch','Lamb Haryali','Lamb Karahi','Lamb Mysore','Lamb Rezala','Lamb Roshni','Lamb Shashlick','Lamb Shashlick Curry',
 'Lamb Sylhet','Lamb Tikka','Lamb Tikka (Main)','Lamb Tikka Balti','Lamb Tikka Biryani','Lamb Tikka Chilli Masala',
 'Lamb Tikka Garlic','Lamb Tikka Jalfrezi','Lamb Tikka Jeera','Lamb Tikka Karahi','Lamb Tikka Masala',
 'Lamb Tikka Pasanda','Lamb Tikka Sizzler','Meat Samosa','Pathia','Persian Chicken Biryani',
 'Persian Lamb Biryani','Prawn Balti','Prawn Biryani','Prawn Karahi','Prawn Puree','Rogon','Royal Paneer','Sheek Kebab',
 'Sheek Kebab (Main)','Tandoori Chicken (1/4)','Tandoori Chicken (Main)','Tandoori Chicken Masala',
 'Tandoori Fish','Tandoori Fish (Main)','Tandoori King Prawn Garlic','Tandoori King Prawn Masala',
 'Tandoori Mixed Grill','Vindaloo','Vindaloo Sauce']
beverages = ['COBRA (330ML)','COBRA (660ML)','Coke 1.5 ltr','Diet Coke 1.5 ltr','House Red wine 75cl',
 'House white wine 75cl','Lemonade 1.5 ltr','Prier Water (750ml)','Still Water (750ml)']
sidedish = ['Chapati','Garlic Naan','Garlic Rice',
 'Green Salad','Bengal Salad','Keema Naan','Keema Rice','Kulcha Naan','Lemon Rice','Lime Pickle','Paratha',
 'Peshwari Naan','Pilau Rice','Plain Naan','Plain Papadum','Plain Rice','Puree','Raitha',
 'Red Sauce','French Fries','Special Fried Rice','Spicy Papadum','Stuffed Paratha','Tandoori Roti','Saag Rice']
data = pd.DataFrame(data) 
type1 = []
for i in data.itertuples():
    if i._2 in mcveg:
        val = 'mcveg'
    if i._2 in mcnonveg:
        val = 'mcnonveg'
    if i._2 in beverages:
        val = 'beverages'
    if i._2 in sidedish:
        val = 'sidedish'
    type1.append(val)
data['type'] = type1     

orders = data
orders.head()
orders.reset_index(inplace=True)
orderdesc = orders.groupby('Order Number')['Item Name'].apply(list).reset_index(name='Order')
del orders

orders = data
orders.reset_index(inplace=True)
ordertype = orders.groupby('Order Number')['type'].apply(list).reset_index(name='Type')
del orders

orders = data
quantamt = orders.groupby(['Order Number']).sum()
quantamt = quantamt[['Quantity','Amount']]
quantamt.reset_index(inplace=True)
del orders

order_date = data[['Order Number','Time','Date']]

complete_data = reduce(lambda x,y: pd.merge(x,y, on='Order Number', how='outer'), [order_date,orderdesc,ordertype])
complete_data = complete_data.drop_duplicates('Order Number')
complete_data = complete_data.reset_index()
complete_data = complete_data.drop('index',axis=1)
hours = pd.to_datetime(complete_data['Time'], format='%H:%M:%S').dt.hour
complete_data['TIMEZONES'] = pd.cut(hours, 
                   bins=[0,6,9,12,15,16,17,18,19,20,21,24], 
                    include_lowest=True, 
                    labels=['6-9 AM','9-12 AM','12-3 PM','3-4 PM','4-5 PM','5-6 PM',
                            '6-7 PM','7-8 PM','8-9 PM','9-12 PM','12-6 AM'])

complete_data1 = reduce(lambda x,y: pd.merge(x,y, on='Order Number', how='outer'), [order_date,quantamt])
complete_data1 = complete_data1.drop_duplicates('Order Number')
complete_data1 = complete_data1.reset_index()
complete_data1 = complete_data1.drop('index',axis=1)
complete_data1['Orders'] = 1
hours = pd.to_datetime(complete_data1['Time'], format='%H:%M:%S').dt.hour
complete_data1['TIMEZONES'] = pd.cut(hours, 
                   bins=[0,6,9,12,15,16,17,18,19,20,21,24], 
                    include_lowest=True, 
                    labels=['6-9 AM','9-12 AM','12-3 PM','3-4 PM','4-5 PM','5-6 PM',
                            '6-7 PM','7-8 PM','8-9 PM','9-12 PM','12-6 AM'])
del data

Wall time: 668 ms


# USER DRIVEN PROGRAM:

- Obtain the starting and ending dates from the user.
- Filter the data according to the user input.
- Define all the functions.(Algorithm of the functions provided in the documentation)
- Call the functions by feeding in the data.

In [5]:
%%time
#ENTER START AND END DATES
print('---------------------------------------------------------------------------------------------------------------')
print('INPUT FORMAT: 2017-MM-DD (AS DATA CONTAINS ONLY ORDERS IN THE YEAR 2017)')
startdate = str(input('ENTER START DATE: '))
enddate = str(input('ENTER END DATE: '))
print('---------------------------------------------------------------------------------------------------------------')
data1 = complete_data1[(complete_data1['Date'] > startdate) & (complete_data1['Date'] < enddate)]
data2 = complete_data[(complete_data['Date'] > startdate) & (complete_data['Date'] < enddate)]
print('PLEASE WAIT AS WE PROCESS YOUR QUERY.....')
for i in range(10):
    print('...')

def getweeklyanalysis(df):
    df = df[['Date','Quantity','Amount','Orders']]
    df['Week'] = df['Date'].dt.week
    df = df.groupby(['Week']).sum()
    df['Averageno_items_per_order'] = df['Quantity']/df['Orders']
    df = df.reset_index()
    filter=0
    while(filter!=4):
        print("\nWEEKLY EARNING ANALYSIS: ")
        print('---------------------------------------------------------------------------------------------------------------')
        filter = int(input("FILTER BY: \n1.AMOUNT(ENTER 1)\n2.ORDERS(ENTER 2)\n3.NO.OF ITEMS PER ORDER(ENTER 3)\n4.SKIP THIS CHART(ENTER 4)\nYOU ENTERED: "))
        print('---------------------------------------------------------------------------------------------------------------')
        if filter== 1:
            fig = px.bar(df, x='Week', y='Amount',
                 color='Amount',color_continuous_scale=px.colors.sequential.Viridis,
                 height=400)
            fig.show()
        elif filter== 2:
            fig = px.bar(df, x='Week', y='Orders',
                 color='Amount',color_continuous_scale=px.colors.sequential.Viridis,
                 height=400)
            fig.show()
        elif filter== 3:
            fig = px.bar(df, x='Week', y='Averageno_items_per_order',
                 color='Amount',color_continuous_scale=px.colors.sequential.Viridis,
                 height=400)
            fig.show()
        else:
            pass
        
def getmonthlyanalysis(df):
    df = df[['Date','Quantity','Amount','Orders']]
    df['Month'] = df['Date'].dt.month
    df = df.groupby(['Month']).sum()
    df['Averageno_items_per_order'] = df['Quantity']/df['Orders']
    df = df.reset_index()
    filter=0
    while(filter!=4):
        print("MONTHLY EARNING ANALYSIS: ")
        print('---------------------------------------------------------------------------------------------------------------')
        filter = int(input("FILTER BY: \n1.AMOUNT(ENTER 1)\n2.ORDERS(ENTER 2)\n3.NO.OF ITEMS PER ORDER(ENTER 3)\n4.SKIP THIS CHART(ENTER 4)\nYOU ENTERED: "))
        print('---------------------------------------------------------------------------------------------------------------')
        if filter== 1:
            fig = px.bar(df, x='Month', y='Amount',
                 color='Amount',color_continuous_scale=px.colors.sequential.Viridis,
                 height=400)
            fig.show()
        elif filter== 2:
            fig = px.bar(df, x='Month', y='Orders',
                 color='Amount',color_continuous_scale=px.colors.sequential.Viridis,
                 height=400)
            fig.show()
        elif filter== 3:
            fig = px.bar(df, x='Month', y='Averageno_items_per_order',
                 color='Amount',color_continuous_scale=px.colors.sequential.Viridis,
                 height=400)
            fig.show()
        else:
            pass
        
def getdailyanalysis(df):
    df = df[['Date','Quantity','Amount','Orders']]
    df = df.groupby(['Date']).sum()
    df['Averageno_items_per_order'] = df['Quantity']/df['Orders']
    df = df.reset_index()
    data = px.data.gapminder()
    filter=0
    while(filter!=4):
        print("DAILY EARNING ANALYSIS: ")
        print('---------------------------------------------------------------------------------------------------------------')
        filter = int(input("FILTER BY: \n1.AMOUNT(ENTER 1)\n2.ORDERS(ENTER 2)\n3.NO.OF ITEMS PER ORDER(ENTER 3)\n4.SKIP THIS CHART(ENTER 4)\nYOU ENTERED: "))
        print('---------------------------------------------------------------------------------------------------------------')
        if filter== 1:
            fig = px.bar(df, x='Date', y='Amount',
                 color='Amount',color_continuous_scale=px.colors.sequential.Viridis,
                 height=400)
            fig.show()
        elif filter== 2:
            fig = px.bar(df, x='Date', y='Orders',
                 color='Amount',color_continuous_scale=px.colors.sequential.Viridis,
                 height=400)
            fig.show()
        elif filter== 3:
            fig = px.bar(df, x='Date', y='Averageno_items_per_order',
                 color='Amount',color_continuous_scale=px.colors.sequential.Viridis,
                 height=400)
            fig.show()
        else:
            pass
        
def gethourlyorders(df):
    a = int(input('DO YOU WANT TO VIEW THE PLOT?\n1.YES(ENTER 1)\n2.NO(ENTER 2)\nENTER YOUR INPUT: '))
    if a==1:
        df = df[['Date','Quantity','Amount','TIMEZONES']]
        grouped_df = df.groupby( [ "Date", "TIMEZONES"])
        df = pd.DataFrame(grouped_df.size().reset_index(name = "ORDERCOUNT"))
        df = df.groupby(['TIMEZONES']).sum()
        df = df.reset_index()
        fig = px.bar(df, x='TIMEZONES', y='ORDERCOUNT',
            color='ORDERCOUNT',color_continuous_scale=px.colors.sequential.Viridis,
            height=400)
        fig.show()
    else:
        pass
    
def getmostordereditem(df):
    print('AVAILABLE TIME ZONES:\n')
    print(' 1.  6-9 AM\n','2.  9-12 AM\n','3.  12-3 PM\n','4.  3-4 PM\n','5.  4-5 PM\n','6.  5-6 PM\n',
      '7.  6-7 PM\n','8.  7-8 PM\n','9.  8-9 PM\n','10. 9-12 PM\n','11. 12-6 AM\n')
    print('ENTER THE SERIAL NO. GIVEN IN FRONT OF THE TIME ZONE: ')
    print('---------------------------------------------------------------------------------------------------------------')
    zone = int(input('ENTER YOUR INPUT: '))
    print('---------------------------------------------------------------------------------------------------------------')
    if zone==1:
        zone = '6-9 AM'
    if zone==2:
        zone = '9-12 AM'
    if zone==3:
        zone = '12-3 PM'
    if zone==4:
        zone = '3-4 PM'
    if zone==5:
        zone = '4-5 PM'
    if zone==6:
        zone = '5-6 PM'
    if zone==7:
        zone = '6-7 PM'
    if zone==8:
        zone = '7-8 PM'
    if zone==9:
        zone = '8-9 PM'
    if zone==10:
        zone = '9-12 PM'
    if zone==11:
        zone = '12-6 AM'
    zonedata = df[df['TIMEZONES']==str(zone)]
    zd = zonedata[['Order','TIMEZONES']]
    zdtype = zd.groupby('TIMEZONES')['Order'].apply(list).reset_index(name='Type')
    value = zdtype[zdtype['TIMEZONES']==str(zone)]
    value.reset_index(inplace=True)
    time_items = value['Type'][0]
    if len(time_items) == 0: 
        print('SEEMS LIKE THEIRS NO ORDER HAVING {} BETWEEN {}.'.format(target,zone))
    else:
        pass
    total_items = reduce(operator.concat, time_items)
    item_distinct = {x:1 for x in total_items}.keys()
    dish=[]
    occ =[]
    for i in item_distinct:
        x = i
        d = Counter(total_items) 
        dish.append(x)
        occ.append(d[x])
    most_occ_item = pd.DataFrame(columns = ['Item Name','Occurence'])
    most_occ_item['Item Name'] = dish
    most_occ_item['Occurence'] = occ
    most_occ_item = most_occ_item.sort_values('Occurence',ascending=False)
    type1 = []
    for i in most_occ_item.itertuples():
        if i._1 in mcveg:
            val = 'mcveg'
        if i._1 in mcnonveg:
            val = 'mcnonveg'
        if i._1 in beverages:
            val = 'beverages'
        if i._1 in sidedish:
            val = 'sidedish'
        type1.append(val)
    most_occ_item['type'] = type1  
    most_occ_item = most_occ_item[(most_occ_item['type']!='sidedish')&(most_occ_item['type']!='beverages')]
    a = most_occ_item[:20]
    del most_occ_item
    a.reset_index(inplace=True)
    a = a.drop('index',axis=1)
    return a

def getcombo(df):
    print('---------------------------------------------------------------------------------------------------------------')
    print('AVAILABLE TIME ZONES:\n')
    print(' 1.  6-9 AM\n','2.  9-12 AM\n','3.  12-3 PM\n','4.  3-4 PM\n','5.  4-5 PM\n','6.  5-6 PM\n',
      '7.  6-7 PM\n','8.  7-8 PM\n','9.  8-9 PM\n','10. 9-12 PM\n','11. 12-6 AM\n')
    print('ENTER THE SERIAL NO. GIVEN IN FRONT OF THE TIME ZONE: ')
    zone = int(input('ENTER YOUR INPUT: '))
    print('---------------------------------------------------------------------------------------------------------------')
    if zone==1:
        zone = '6-9 AM'
    if zone==2:
        zone = '9-12 AM'
    if zone==3:
        zone = '12-3 PM'
    if zone==4:
        zone = '3-4 PM'
    if zone==5:
        zone = '4-5 PM'
    if zone==6:
        zone = '5-6 PM'
    if zone==7:
        zone = '6-7 PM'
    if zone==8:
        zone = '7-8 PM'
    if zone==9:
        zone = '8-9 PM'
    if zone==10:
        zone = '9-12 PM'
    if zone==11:
        zone = '12-6 AM'
    combo_data = df[df['TIMEZONES']==str(zone)]
    type2 = []
    print("AVAILABLE SIDE DISHES FOR MAKING COMBINATIONS WITH: \nChapati\nGarlic Naan\nGarlic Rice\nGreen Salad\nBengal Salad\nKeema Naan\nKeema Rice\nKulcha Naan\nLemon Rice\nLime Pickle\nParatha\nPeshwari Naan\nPilau Rice\nPlain Naan\nPlain Papadum\nPlain Rice\nPuree\nRaitha\nRed Sauce\nFrench Fries\nSpecial Fried Rice\nSpicy Papadum\nStuffed Paratha\nTandoori Roti\nSaag Rice")
    target = input("ENTER THE SIDE DISH FOR WHICH MAIN COURSE COMBO'S ARE TO BE ANALYSED: ")
    for i in combo_data.itertuples():
        if target in i.Order:
            val = 'Found'
        else:
            val = 'Not found'
        type2.append(val)
    combo_data['Status'] = type2
    combo_data = combo_data[combo_data['Status']=='Found']
    combo1 = combo_data[['Order','TIMEZONES']]
    ordertype1 = combo1.groupby('TIMEZONES')['Order'].apply(list).reset_index(name='Type')
    val = ordertype1[ordertype1['TIMEZONES']==str(zone)]
    val.reset_index(inplace=True)
    count = val['Type'][0]
    if len(count) == 0: 
        print('SEEMS LIKE THEIRS NO ORDER HAVING {} BETWEEN {}.'.format(target,zone))
    else:
        total_item = reduce(operator.concat, count)
        item_distinct = {x:1 for x in total_item}.keys()
        dish=[]
        occ =[]
        for i in item_distinct:
            x = i
            d = Counter(total_item) 
            dish.append(x)
            occ.append(d[x])
        column = 'Ordered with {} between {}'.format(target,zone)
        combo_final = pd.DataFrame(columns = ['Item Name',column])
        combo_final['Item Name'] = dish
        combo_final[column] = occ
        combo_final = combo_final.sort_values(column,ascending=False)
        type1 = []
        for i in combo_final.itertuples():
            if i._1 in mcveg:
                val = 'mcveg'
            if i._1 in mcnonveg:
                val = 'mcnonveg'
            if i._1 in beverages:
                val = 'beverages'
            if i._1 in sidedish:
                val = 'sidedish'
            type1.append(val)
        combo_final['type'] = type1  
        combo_final = combo_final[combo_final['type']!='sidedish']
        return combo_final[:10]
print('---------------------------------------------------------------------------------------------------------------')
getweeklyanalysis(data1)
getmonthlyanalysis(data1)
getdailyanalysis(data1)
print("HERE IS THE TIME WHEN MOST OF YOUR ORDERS ARE RECIEVED: ")
gethourlyorders(data1)
print('---------------------------------------------------------------------------------------------------------------')
print("HERE ARE THE MOST ORDERED ITEMS IN A GIVEN TIME ZONE: ")
c = getmostordereditem(data2)
print("MOST ORDERED ITEMS: ")
print('---------------------------------------------------------------------------------------------------------------')
display(c)
print('---------------------------------------------------------------------------------------------------------------')
print("FIND COMBINATIONS ACCORDING TO TIME ZONES IN A DAY: ")
a = getcombo(data2)
a

---------------------------------------------------------------------------------------------------------------
INPUT FORMAT: 2017-MM-DD (AS DATA CONTAINS ONLY ORDERS IN THE YEAR 2017)
ENTER START DATE: 2017-02-01
ENTER END DATE: 2017-08-01
---------------------------------------------------------------------------------------------------------------
PLEASE WAIT AS WE PROCESS YOUR QUERY.....
...
...
...
...
...
...
...
...
...
...
---------------------------------------------------------------------------------------------------------------

WEEKLY EARNING ANALYSIS: 
---------------------------------------------------------------------------------------------------------------
FILTER BY: 
1.AMOUNT(ENTER 1)
2.ORDERS(ENTER 2)
3.NO.OF ITEMS PER ORDER(ENTER 3)
4.SKIP THIS CHART(ENTER 4)
YOU ENTERED: 2
---------------------------------------------------------------------------------------------------------------



WEEKLY EARNING ANALYSIS: 
---------------------------------------------------------------------------------------------------------------
FILTER BY: 
1.AMOUNT(ENTER 1)
2.ORDERS(ENTER 2)
3.NO.OF ITEMS PER ORDER(ENTER 3)
4.SKIP THIS CHART(ENTER 4)
YOU ENTERED: 4
---------------------------------------------------------------------------------------------------------------
MONTHLY EARNING ANALYSIS: 
---------------------------------------------------------------------------------------------------------------
FILTER BY: 
1.AMOUNT(ENTER 1)
2.ORDERS(ENTER 2)
3.NO.OF ITEMS PER ORDER(ENTER 3)
4.SKIP THIS CHART(ENTER 4)
YOU ENTERED: 2
---------------------------------------------------------------------------------------------------------------


MONTHLY EARNING ANALYSIS: 
---------------------------------------------------------------------------------------------------------------
FILTER BY: 
1.AMOUNT(ENTER 1)
2.ORDERS(ENTER 2)
3.NO.OF ITEMS PER ORDER(ENTER 3)
4.SKIP THIS CHART(ENTER 4)
YOU ENTERED: 4
---------------------------------------------------------------------------------------------------------------
DAILY EARNING ANALYSIS: 
---------------------------------------------------------------------------------------------------------------
FILTER BY: 
1.AMOUNT(ENTER 1)
2.ORDERS(ENTER 2)
3.NO.OF ITEMS PER ORDER(ENTER 3)
4.SKIP THIS CHART(ENTER 4)
YOU ENTERED: 2
---------------------------------------------------------------------------------------------------------------


DAILY EARNING ANALYSIS: 
---------------------------------------------------------------------------------------------------------------
FILTER BY: 
1.AMOUNT(ENTER 1)
2.ORDERS(ENTER 2)
3.NO.OF ITEMS PER ORDER(ENTER 3)
4.SKIP THIS CHART(ENTER 4)
YOU ENTERED: 4
---------------------------------------------------------------------------------------------------------------
HERE IS THE TIME WHEN MOST OF YOUR ORDERS ARE RECIEVED: 
DO YOU WANT TO VIEW THE PLOT?
1.YES(ENTER 1)
2.NO(ENTER 2)
ENTER YOUR INPUT: 1


---------------------------------------------------------------------------------------------------------------
HERE ARE THE MOST ORDERED ITEMS IN A GIVEN TIME ZONE: 
AVAILABLE TIME ZONES:

 1.  6-9 AM
 2.  9-12 AM
 3.  12-3 PM
 4.  3-4 PM
 5.  4-5 PM
 6.  5-6 PM
 7.  6-7 PM
 8.  7-8 PM
 9.  8-9 PM
 10. 9-12 PM
 11. 12-6 AM

ENTER THE SERIAL NO. GIVEN IN FRONT OF THE TIME ZONE: 
---------------------------------------------------------------------------------------------------------------
ENTER YOUR INPUT: 7
---------------------------------------------------------------------------------------------------------------
MOST ORDERED ITEMS: 
---------------------------------------------------------------------------------------------------------------


,Item Name,Occurence,type
0,Korma,104,mcnonveg
1,Onion Bhajee,100,mcveg
2,Chicken Tikka Masala,97,mcnonveg
3,Mango Chutney,90,mcveg
4,Bombay Aloo,71,mcveg
5,Madras,71,mcveg
6,Mushroom Rice,58,mcveg
7,Meat Samosa,53,mcnonveg
8,Mint Sauce,52,mcveg
9,Bhuna,47,mcveg


---------------------------------------------------------------------------------------------------------------
FIND COMBINATIONS ACCORDING TO TIME ZONES IN A DAY: 
---------------------------------------------------------------------------------------------------------------
AVAILABLE TIME ZONES:

 1.  6-9 AM
 2.  9-12 AM
 3.  12-3 PM
 4.  3-4 PM
 5.  4-5 PM
 6.  5-6 PM
 7.  6-7 PM
 8.  7-8 PM
 9.  8-9 PM
 10. 9-12 PM
 11. 12-6 AM

ENTER THE SERIAL NO. GIVEN IN FRONT OF THE TIME ZONE: 
ENTER YOUR INPUT: 7
---------------------------------------------------------------------------------------------------------------
AVAILABLE SIDE DISHES FOR MAKING COMBINATIONS WITH: 
Chapati
Garlic Naan
Garlic Rice
Green Salad
Bengal Salad
Keema Naan
Keema Rice
Kulcha Naan
Lemon Rice
Lime Pickle
Paratha
Peshwari Naan
Pilau Rice
Plain Naan
Plain Papadum
Plain Rice
Puree
Raitha
Red Sauce
French Fries
Special Fried Rice
Spicy Papadum
Stuffed Paratha
Tandoori Roti
Saag Rice
ENTER THE SIDE DISH FOR WHICH M

,Item Name,Ordered with Plain Naan between 6-7 PM,type
0,Korma,45,mcnonveg
17,Chicken Tikka Masala,44,mcnonveg
40,Mango Chutney,35,mcveg
7,Onion Bhajee,32,mcveg
13,Madras,24,mcveg
23,Curry,22,mcveg
31,Bhuna,20,mcveg
21,Bombay Aloo,19,mcveg
1,Dhansak,18,mcveg
16,Saag Aloo,17,mcveg
